In [1]:
# Import necessary libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
import os

# Import CRUD module and setup database connection
from crud import AnimalShelter

# User credentials
username = "aacuser"
password = "SNHU1234"

# Initialize database connection
db = AnimalShelter(username, password)

# Fetch all records from MongoDB
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)  # Drop MongoDB ObjectID column

# Setup the Jupyter version of Dash
app = JupyterDash(__name__)

# Encode and insert the Grazioso Salvare logo
image_filename = 'my-image.png'  # Update with correct file path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define app layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('CS-340 Dashboard by Brad Mills'))),
    html.Hr(),
    
    # Interactive filter options
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster or Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        )
    ]),
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Pagination control
        sort_action="native",  # Enables sorting
        filter_action="native",  # Enables filtering
        row_selectable="single",  # Allow selection of a single row
        selected_rows=[]  # Initialize with no row selected
    ),
    html.Br(),
    html.Hr(),
    
    # Layout for charts side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to update data table based on filter selection
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
    elif filter_type == 'Mountain or Wilderness':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'Disaster or Tracking':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}  # Reset filter
    
    filtered_data = pd.DataFrame.from_records(db.read(query))
    filtered_data.drop(columns=['_id'], inplace=True)  # Remove MongoDB ObjectID column if present
    return filtered_data.to_dict('records')

# Callback for breed distribution chart
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Animal Breeds')
    return [dcc.Graph(figure=fig)]

# Callback for geolocation map based on selected data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "selected_rows")]
)
def update_map(viewData, selected_rows):
    if viewData is None or not selected_rows:
        return []
    
    # Get the selected row data
    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]  # Only a single row is selected

    # Extract latitude and longitude of the selected animal
    latitude = dff.iloc[row]['location_lat']  # Adjust if your latitude column name differs
    longitude = dff.iloc[row]['location_long']  # Adjust if your longitude column name differs
    breed = dff.iloc[row]['breed']
    animal_name = dff.iloc[row]['name']  # Adjust column name if different

    # Austin TX coordinates as the map center
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(breed),  # Animal breed
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)  # Animal name
                ])
            ])
        ])
    ]

# Highlight selected data table cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Run app server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:13444/
